In [33]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter

url = 'http://www.tides.gc.ca/C&A/network_means-eng.html#'
response = requests.get(url)

# checks if access was successful 
response

<Response [200]>

In [41]:
soup = BeautifulSoup(response.text, "html.parser")
mean_tables = soup.findAll('table', {'class':'span-8'})
alltime_tables = soup.findAll('table', {'title':'All-Time'})
montreal_mean_table = soup.find('table', {'title':'Montreal Harbour monthly mean water levels in metres referred to IGLD 1985'})

In [42]:
def rowsDF(table):
    
    row_vals = []
    row_links = table.findAll('tr')
    
    for tr in row_links:
        tds = tr.findAll('td')
        row_vals.append(tds)
    
    for i in range(len(row_vals)):
        for j in range(len(row_vals[i])):
            row_vals[i][j] = row_vals[i][j].text
        
    data_set = pd.DataFrame(row_vals)
    return data_set

def row_labels(table):
    label_links = table.findAll('th')
    
    labels = []
    for i in range(len(label_links)): 
        label_links[i] = label_links[i].text
        labels.append(label_links[i])
        
    return labels

In [43]:
# add labels for the data frame in each lake

def meanDF(table):
    
    data_frame = rowsDF(table)
    data_frame.columns = row_labels(table)
    data_frame = data_frame.dropna()
    
    return data_frame

def alltimeDF(table):
    
    data_frame = rowsDF(table)
    data_frame.index = row_labels(table)
    
    return data_frame

In [44]:
# create list of dataframes for each data set

data_set1 = []
for i in range(len(mean_tables)):
    data_set1.append(meanDF(mean_tables[i]))
data_set1.append(meanDF(montreal_mean_table)) 

data_set2 = []
for i in range(len(alltime_tables)):
    data_set2.append(alltimeDF(alltime_tables[i]))
    

In [50]:
path1 = r'C:\Users\tania\Documents\TDW Tech Test\dataset1.xlsx'
path2 = r'C:\Users\tania\Documents\TDW Tech Test\dataset2.xlsx'

def save_file(data_set, path, boolean):
    with ExcelWriter(path) as writer:
        for n, df in enumerate(data_set):
            df.to_excel(writer,'sheet%s' % n, index=boolean)
        writer.save()
        
save_file(data_set1, path1, False)
save_file(data_set2, path2, True)